In [ ]:
!unzip Headshots.zip -d Headshots

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = \
    train_datagen.flow_from_directory('./Headshots',
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=True)

Found 224 images belonging to 2 classes.


In [4]:
train_generator.class_indices.values()
# dict_values([0, 1, 2])
NO_CLASSES = len(train_generator.class_indices.values())

In [5]:
!pip install keras_vggface

In [6]:
!pip install keras_applications
!pip install keras_preprocessing

In [8]:
from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=True,
    model='vgg16',
    input_shape=(224, 224, 3))
base_model.summary()

print(len(base_model.layers))
# 26 layers in the original VGG-Face


580070376/580070376 [==============================] - 4s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                    

In [9]:
 
base_model = VGGFace(include_top=False,
model='vgg16',
input_shape=(224, 224, 3))
base_model.summary()
print(len(base_model.layers))
# 19 layers after excluding the last few layers

58909280/58909280 [==============================] - 1s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                      

In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# final layer with softmax activation
preds = Dense(NO_CLASSES, activation='softmax')(x)
model = Model(base_model.input,preds)

In [17]:

# don't train the first 19 layers - 0..18
for layer in model.layers[:19]:
    layer.trainable = False

# train the rest of the layers - 19 onwards
for layer in model.layers[19:]:
    layer.trainable = True

In [18]:
model.compile(optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [19]:
model.fit(train_generator,
  batch_size = 1,
  verbose = 1,
  epochs = 20)

Epoch 1/20
7/7 [==============================] - 2s 166ms/step - loss: 0.5797 - accuracy: 0.7098
Epoch 2/20
7/7 [==============================] - 1s 166ms/step - loss: 0.2125 - accuracy: 0.9152
Epoch 3/20
7/7 [==============================] - 1s 169ms/step - loss: 0.1712 - accuracy: 0.9509
Epoch 4/20
7/7 [==============================] - 1s 169ms/step - loss: 0.0904 - accuracy: 0.9866
Epoch 5/20
7/7 [==============================] - 1s 168ms/step - loss: 0.0566 - accuracy: 0.9911
Epoch 6/20
7/7 [==============================] - 1s 170ms/step - loss: 0.0616 - accuracy: 0.9866
Epoch 7/20
7/7 [==============================] - 1s 173ms/step - loss: 0.0889 - accuracy: 0.9866
Epoch 8/20
7/7 [==============================] - 1s 170ms/step - loss: 0.0455 - accuracy: 0.9866
Epoch 9/20
7/7 [==============================] - 1s 170ms/step - loss: 0.0389 - accuracy: 0.9911
Epoch 10/20
7/7 [==============================] - 1s 171ms/step - loss: 0.0389 - accuracy: 0.9911
Epoch 11/20
7/7 [==

In [20]:
# creates a HDF5 file
model.save(
    'transfer_learning_trained' +
    '_face_cnn_model.h5')


In [15]:
from tensorflow.keras.models import load_model

model = load_model(
    'transfer_learning_trained' +
    '_face_cnn_model.h5')


In [23]:
from PIL import Image

In [28]:
face_to_check = Image.open(str('/content/0_picture_3.png'))
face_to_check = face_to_check.resize((224,224))
face_to_check = np.array(face_to_check)

In [30]:
model = keras.models.load_model('transfer_learning_trained_face_cnn_model.h5')
# model = keras.models.load_model('face_predictor2.0')
preds = model.pr(x=np.array([face_to_check]))
print(preds[0])

AttributeError: ignored